In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
%matplotlib inline
from matplotlib import pyplot as plt
from keras.callbacks import CSVLogger
import gc


config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

root = os.getcwd()


In [2]:
def data_generator(encoded_images_array, ground_truth_array, start_index, batch_size):
    
    path_post_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\post_enc_frames"
    path_pre_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\pre_enc_frames"
    
    folders = os.listdir(path_post_enc)
    for i in range(start_index, start_index + batch_size):
        os.chdir(os.path.join(path_post_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                encoded_images_array.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256, 1)))

    folders = os.listdir(path_pre_enc)
    for i in range(start_index, start_index + batch_size):
        os.chdir(os.path.join(path_pre_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                ground_truth_array.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256, 1)))

    return start_index + batch_size


In [3]:
def validation_data(X_val, y_val, val_size=15, start_idx=120):
    path_post_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\post_enc_frames"
    path_pre_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\pre_enc_frames"
    
    folders = os.listdir(path_post_enc)
    for i in range(start_idx, start_idx+val_size):
        os.chdir(os.path.join(path_post_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                X_val.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

    folders = os.listdir(path_pre_enc)
    for i in range(start_idx, start_idx+val_size):
        os.chdir(os.path.join(path_pre_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                y_val.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256)))

In [4]:
def SpatialAttention(input):
     x = layers.Conv2D(filters=8, kernel_size=3, strides=1, padding='same')(input)
     x = layers.ReLU()(x)
     x = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(x)
     out = layers.Activation('sigmoid')(x)

     return out

In [5]:

def ChannelAttention(input):
    sh = input.get_shape().as_list()
    x = layers.GlobalAveragePooling2D()(input)
    x = layers.Dense(8)(x)
    x = layers.ReLU()(x)
    x = layers.Dense(sh[-1])(x)
    out = layers.Activation('sigmoid')(x)

    return out


In [6]:

def residual_block(input):
     x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(input)
     #x_c = ChannelAttention(x)
     x_s = SpatialAttention(x)
     #x1 = Multiply()([x,x_c]) #Broadcast
     x2 = layers.Multiply()([x,x_s]) #Broadcast
     #x3 = Concatenate()([x1,x2])
     x2 = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x2)
     x2 = layers.LeakyReLU(0.2)(x2)
     x2 = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x2)
     out=layers.Add()([x2,input])
     return out

In [7]:
def attention_net(input_shape=(None, None, 1), filters=64,  
name='Attention_net'):
     lr_image=layers.Input(shape=input_shape, name='input')
     x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(lr_image)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     x=residual_block(x)
     out=layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(x)
     return tf.keras.Model(inputs=lr_image, outputs=out, name=name)

In [8]:
num_blocks = 10
Model = attention_net()

In [9]:
csv_logger = CSVLogger('C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\log.csv', append=True, separator=';')

In [10]:
checkpoint_filepath = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\chkpt\\model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='max',
    save_best_only=False,
    save_weights_only=False,
    save_freq="epoch")

In [11]:
Model = tf.keras.models.load_model(checkpoint_filepath)

In [12]:
optimizer = optimizers.Adam(learning_rate=1e-4, beta_1=0, beta_2=0.999)
Model.compile(optimizer=optimizer, loss='mse', metrics=["mse", "mae"])
# Model.summary()

In [13]:
Model.summary()

Model: "Attention_net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   320         ['input[0][0]']                  
                                32)                                                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, None, None,   9248        ['conv2d[0][0]']                 
                                32)                                                   

In [14]:
# epochs = 460 (0.8 test size)

epochs = 15
for i in range (60):
    file_path = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\devam_indisi.txt'
    with open(file_path, 'r') as file:
        st_ix = int(file.read())
        
    X_train = []
    y_train = []
    st_ix = data_generator(X_train, y_train, st_ix, 8)
    if(st_ix == 160):
        st_ix = 0
        
    
    X_train = np.array(X_train) / 255.0
    y_train = np.array(y_train) / 255.0

    Model.fit(
        x=X_train,
        y=y_train,
        epochs=1,
        batch_size=4,
        shuffle=True,
        callbacks=[model_checkpoint_callback, csv_logger]
    )

    gc.collect()

    with open(file_path, 'w') as file:
        file.write(f"{st_ix}")


260/260 [==============================] - 111s 424ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0189


In [23]:
X_val = []
y_val = []
validation_data(X_val, y_val, val_size=15, start_idx=190)
X_val = np.array(X_val) / 255.0
y_val = np.array(y_val) / 255.0


In [25]:
gc.collect()

11577

In [26]:

predictions = Model.predict(X_val)
predictions[predictions < 0.0] = 0.0
predictions[predictions > 1.0] = 1.0


62/62 [==============================] - 70s 1s/step


In [27]:
predictions = np.reshape(predictions, (X_val.shape))

pred_loss = np.sum(np.square(predictions - y_val)) / predictions.shape[0]
x_loss = np.sum(np.square(X_val - y_val)) / predictions.shape[0]

print(f"pred loss: {pred_loss}")
print(f"x_loss: {x_loss}")
print(predictions.shape)
print(X_val.shape)
print(predictions.max())

pred loss: 75.4033550574052
x_loss: 77.00635903828774
(1980, 256, 256)
(1980, 256, 256)
1.0


In [19]:

idx = 0
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\X.png", X_val[idx] * 255.0)
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\pred.png", predictions[idx] * 255.0)
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\y.png", y_val[idx] * 255.0)


True